In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('../data/auto.csv', index_col='ID')
print(f"Исходный датафрейм загружен. Размер: {df.shape}")
print(df.head())

Исходный датафрейм загружен. Размер: (931, 5)
       CarNumber  Make_n_model  Refund   Fines  History
ID                                                     
0   Y163O8161RUS    Ford Focus     2.0  3200.0      NaN
1    E432XX77RUS  Toyota Camry     1.0  6500.0      NaN
2    7184TT36RUS    Ford Focus     1.0  2100.0      NaN
3   X582HE161RUS    Ford Focus     2.0  2000.0      NaN
4   E34877152RUS    Ford Focus     2.0  6100.0      NaN


In [3]:
print("\nПодсчет количества наблюдений:")
observation_counts = df.count()
print(observation_counts)
print(f"Общее количество строк: {len(df)}")


Подсчет количества наблюдений:
CarNumber       931
Make_n_model    931
Refund          914
Fines           869
History          82
dtype: int64
Общее количество строк: 931


In [4]:
#Удаление дубликатов по столбцам CarNumber, Make_n_model, Fines
print(f"Размер до удаления дубликатов: {df.shape}")

# Удаляем дубликаты, оставляя последние записи
df_no_duplicates = df.drop_duplicates(subset=['CarNumber', 'Make_n_model', 'Fines'], keep='last')
print(f"Размер после удаления дубликатов: {df_no_duplicates.shape}")

# Проверяем количество наблюдений снова
observation_counts_after = df_no_duplicates.count()
print("Количество наблюдений после удаления дубликатов:")
print(observation_counts_after)

Размер до удаления дубликатов: (931, 5)
Размер после удаления дубликатов: (725, 5)
Количество наблюдений после удаления дубликатов:
CarNumber       725
Make_n_model    725
Refund          713
Fines           665
History          65
dtype: int64


In [5]:
#Работа с пропущенными значениями
#Проверяем количество пропущенных значений в каждом столбце
print("Количество пропущенных значений в каждом столбце:")
missing_values = df_no_duplicates.isnull().sum()
print(missing_values)

Количество пропущенных значений в каждом столбце:
CarNumber         0
Make_n_model      0
Refund           12
Fines            60
History         660
dtype: int64


In [6]:
#Удаляем столбцы с более чем 500 пропущенными значениями
print("\n4.2 Удаление столбцов с более чем 500 пропущенными значениями...\n")
# thresh означает минимальное количество непустых значений, которое должно быть в строке
# Но здесь нам нужно удалить столбцы, поэтому проверим каждый столбец
columns_to_drop = []
for col in df_no_duplicates.columns:
    missing_count = df_no_duplicates[col].isnull().sum()
    if missing_count > 500:
        columns_to_drop.append(col)
        print(f"Столбец '{col}' имеет {missing_count} пропущенных значений и будет удален\n")

if columns_to_drop:
    df_no_duplicates = df_no_duplicates.drop(columns=columns_to_drop)
else:
    print("Нет столбцов с более чем 500 пропущенными значениями")

print("Количество пропущенных значений после удаления столбцов:")
print(df_no_duplicates.isnull().sum())


4.2 Удаление столбцов с более чем 500 пропущенными значениями...

Столбец 'History' имеет 660 пропущенных значений и будет удален

Количество пропущенных значений после удаления столбцов:
CarNumber        0
Make_n_model     0
Refund          12
Fines           60
dtype: int64


In [7]:
# Заполнение пропущенных значений в столбце Refund предыдущим значением
print("\nЗаполнение пропущенных значений в столбце Refund...")
df_no_duplicates['Refund'] = df_no_duplicates['Refund'].ffill()
print("Количество пропущенных значений после заполнения Refund:")
print(df_no_duplicates.isnull().sum())


Заполнение пропущенных значений в столбце Refund...
Количество пропущенных значений после заполнения Refund:
CarNumber        0
Make_n_model     0
Refund           0
Fines           60
dtype: int64


In [8]:
# Заполнение пропущенных значений в столбце Fines средним значением
print("\n4.4 Заполнение пропущенных значений в столбце Fines средним значением...")
fines_mean = df_no_duplicates['Fines'].mean()
print(f"Среднее значение Fines: {fines_mean:.2f}")
df_no_duplicates['Fines'] = df_no_duplicates['Fines'].fillna(fines_mean)
print("Количество пропущенных значений после заполнения Fines:")
print(df_no_duplicates.isnull().sum())


4.4 Заполнение пропущенных значений в столбце Fines средним значением...
Среднее значение Fines: 8594.59
Количество пропущенных значений после заполнения Fines:
CarNumber       0
Make_n_model    0
Refund          0
Fines           0
dtype: int64


In [9]:
# Разделение и парсинг марки и модели
print("\nРазделение Make_n_model на Make и Model...")

# Функция для извлечения марки (первое слово)
def extract_make(make_model):
    if pd.isna(make_model):
        return None
    return make_model.split()[0]

# Функция для извлечения модели (остальные слова)
def extract_model(make_model):
    if pd.isna(make_model):
        return None
    parts = make_model.split()
    if len(parts) > 1:
        return ' '.join(parts[1:])
    else:
        return ''

# Применяем функции для создания новых столбцов
df_no_duplicates['Make'] = df_no_duplicates['Make_n_model'].apply(extract_make)
df_no_duplicates['Model'] = df_no_duplicates['Make_n_model'].apply(extract_model)

print("Первые 5 строк с новыми столбцами Make и Model:")
print(df_no_duplicates[['Make_n_model', 'Make', 'Model']].head())

# Удаляем столбец Make_n_model
df_final = df_no_duplicates.drop('Make_n_model', axis=1)

print("\nФинальная структура данных:")
print(df_final.head())
print(f"\nФинальный размер датафрейма: {df_final.shape}")
print(f"Столбцы: {list(df_final.columns)}")


Разделение Make_n_model на Make и Model...
Первые 5 строк с новыми столбцами Make и Model:
    Make_n_model    Make  Model
ID                             
0     Ford Focus    Ford  Focus
1   Toyota Camry  Toyota  Camry
2     Ford Focus    Ford  Focus
3     Ford Focus    Ford  Focus
5     Ford Focus    Ford  Focus

Финальная структура данных:
       CarNumber  Refund   Fines    Make  Model
ID                                             
0   Y163O8161RUS     2.0  3200.0    Ford  Focus
1    E432XX77RUS     1.0  6500.0  Toyota  Camry
2    7184TT36RUS     1.0  2100.0    Ford  Focus
3   X582HE161RUS     2.0  2000.0    Ford  Focus
5   92918M178RUS     1.0  5700.0    Ford  Focus

Финальный размер датафрейма: (725, 5)
Столбцы: ['CarNumber', 'Refund', 'Fines', 'Make', 'Model']


In [10]:
# Сохранение в JSON файл
print("\nСохранение в JSON файл...")

# Подготавливаем данные для JSON (сбрасываем индекс, чтобы не включать ID)
df_for_json = df_final.reset_index(drop=True)

# Создаем список словарей
json_data = []
for _, row in df_for_json.iterrows():
    record = {
        "CarNumber": row['CarNumber'],
        "Refund": float(row['Refund']) if pd.notna(row['Refund']) else None,
        "Fines": float(row['Fines']) if pd.notna(row['Fines']) else None,
        "Make": row['Make'],
        "Model": row['Model']
    }
    # Удаляем History если он есть
    if 'History' in row.index and pd.notna(row['History']):
        record["History"] = row['History']
    json_data.append(record)

# Сохраняем в JSON файл
with open('auto.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print("Данные успешно сохранены в auto.json")
print(f"Количество записей в JSON: {len(json_data)}")

# Показываем первые 2 записи в формате JSON для проверки
print("\nПример первых 2 записей в JSON формате:")
print(json.dumps(json_data[:2], ensure_ascii=False, indent=2))


Сохранение в JSON файл...
Данные успешно сохранены в auto.json
Количество записей в JSON: 725

Пример первых 2 записей в JSON формате:
[
  {
    "CarNumber": "Y163O8161RUS",
    "Refund": 2.0,
    "Fines": 3200.0,
    "Make": "Ford",
    "Model": "Focus"
  },
  {
    "CarNumber": "E432XX77RUS",
    "Refund": 1.0,
    "Fines": 6500.0,
    "Make": "Toyota",
    "Model": "Camry"
  }
]


In [ ]:
print(df_no_duplicates['Fines'] = df_no_duplicates['Fines'].fillna(fines_mean))